## 3.4 Sparse Matrices and Band Matrices

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

We use the library `scipy` to provide the data structure for sparse matrices. We also use `matplotlib` to visualize the sparsity structure of the matrices.

**Implementation 3.6: Cuthill-McKee Algorithm**

In [ ]:
def cuthill_mckee(A):
    n = A.shape[0]
    row, col = A.nonzero()
    N = [(l, len(l)) for l in [list(row[col == i]) for i in range(n)]]

    I, Q = [], []
    R = [i for i in range(n)]

    for k in range(n):
        if len(I) == n:
            break
        elif len(I) == k:
            i = R[np.argmin(np.array([N[i][1] for i in R]))]
            I.append(i)
            Q.append(i)
            R.remove(i)

        i = Q[0]
        neighbours = [n for n in N[i][0] if n not in I]
        neighbours_sort = sorted(neighbours, key=lambda i: N[i][1])
        for ik in neighbours_sort:
            I.append(ik)
            Q.append(ik)
            R.remove(ik)
        Q.pop(0)

    data, row, col = [], [], []
    for key in A.todok().keys():
        data.append(A[key])
        row.append(I.index(key[0]))
        col.append(I.index(key[1]))

    return sp.sparse.csr_matrix((data, (row, col)), shape=(n, n))

#### Example 3.17 (Cuthill-McKee)
We apply our implementation to the matrix given in the book. For the sake of simplicity, we take 1 as the non-zero value.

In [ ]:
A1 = sp.sparse.csr_matrix([[1, 1, 0, 0, 0, 0, 0, 1],
                           [1, 1, 0, 0, 1, 1, 0, 1],
                           [0, 0, 1, 0, 1, 0, 1, 1],
                           [0, 0, 0, 1, 0, 0, 0, 0],
                           [0, 1, 1, 0, 1, 0, 1, 0],
                           [0, 1, 0, 0, 0, 1, 0, 0],
                           [0, 0, 1, 0, 1, 0, 1, 0],
                           [1, 1, 1, 0, 0, 0, 0, 1]])

plt.spy(A1)
plt.show()

In [ ]:
A2 = cuthill_mckee(A1)
plt.spy(A2)
plt.show()

#### Beispiel 3.18 (Cholesky factorisation for bandmatrices)

From Theorem 3.17 (LU factorization of a band matrix), we know that the LU factorization of a band matrix again produces band matrices with the same bandwidth. We can take advantage of this in the implementation of the Cholesky factorization to avoid unnecessary work.

In [ ]:
def cholesky_band(A):
    n, m = A.shape
    assert n == m, 'Matrix must be square!'
    L = np.zeros_like(A)

    for p in range(n - 1):
        if np.allclose(L, np.triu(A, k=p + 1)) and np.allclose(L, np.tril(A, k=-p - 1)):
            break

    for j in range(0, n):
        ll = 0
        m = max(0, j - p)
        for k in range(m, j):
            ll += L[j, k]**2
        L[j, j] = np.sqrt(A[j, j] - ll)

        for i in range(j + 1, min(j + 1 + p, n)):
            ll = 0
            for k in range(m, j):
                ll += L[i, k] * L[j, k]
            L[i, j] = (A[i, j] - ll) / L[j, j]
    return L

To test this, we begin by generating sparse, positive definite matrices.

In [ ]:
C = sp.sparse.rand(100, 100, density=0.02, format='csr', random_state=100)
X = sp.sparse.eye(100) + C @ C.T

plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, width=0)
plt.spy(X, markersize=2)
plt.show()

If we use the standard Cholesky factorization to this matrix, we see that the resuling matrix $L$ is dense. Even the lowest row has many non-zero entries.

In [ ]:
L = cholesky_band(X.toarray())
plt.spy(sp.sparse.csr_matrix(L), markersize=2)
plt.show()

However, if we first sort the matrix with the Cuthill-McKee algorithm, we obtain a band structure in the matrix which can be taken advantage of in the Cholesky factorization.

In [ ]:
X1 = cuthill_mckee(X)
plt.spy(X1, markersize=2)
plt.show()

Our Cholesky factorization can now take advantage of this, such that $L$ is again a band matrix.

In [ ]:
L = cholesky_band(X1.toarray())
plt.spy(sp.sparse.csr_matrix(L), markersize=2)
plt.show()

Nevertheless, we see that there are many non-zero entries within the band structure. However, we can take advantage of the band structure both in the forward and backward substitution, making these steps significantly cheaper.